# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date (leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(0,"","",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-21T21:31:00 to 2019-02-22T21:31:00
152


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

100%|██████████| 94/94 [00:00<00:00, 1842.30it/s]

Binary Encoding


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

                                               title  \
0                       What price pension freedoms?   
1  The Daily 202: Will supporting reparations bec...   
2    Windstream debt battle opens up ‘Pandora’s Box’   
3  Kraft Heinz shares fall 28 percent after write...   
4  Boeing, Goldman and three other Dow stocks are...   

                                         description  
0  George Osborne’s key policy had unintended con...  
1  Four presidential candidates have now expresse...  
2  Arkansas-based telecoms company now in trouble...  
3  Kraft Heinz Co shares fell to a record low on ...  
4  Just five of the stocks of the Dow 30 have car...  


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

[nltk_data] Downloading package punkt to /Users/jadekhiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /Users/jadekhiev/nltk_data...
[nltk_data]   Package brown is already up-to-date!


100%|██████████| 94/94 [00:08<00:00, 11.73it/s]


In [6]:
import Recommender as rec
articleDB = rec.recommender(binaryMatrix, articleDB)

In [7]:
articleDB

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,cleanContent,tags,tags_top_5,related_articles
0,0.364035,0.635965,https://www.ft.com/content/b5151486-3395-11e9-...,1,What price pension freedoms?,George Osborne’s key policy had unintended con...,financial-times,2019-02-22T05:01:00Z,The secret to happiness is freedom wrote the ...,"The secret to happiness is freedom, wrote the ...","The secret to happiness is freedom, wrote the ...","uk pensioners, adult children, high price, exp...","good decisions,place admission,also underspend...","83, 56, 78"
1,0.407633,0.592367,https://www.washingtonpost.com/news/powerpost/...,1,The Daily 202: Will supporting reparations bec...,Four presidential candidates have now expresse...,the-washington-post,2019-02-22T15:05:54Z,Sen Elizabeth Warren D Mass campaigns in La...,Sen. Elizabeth Warren (D-Mass.) campaigns in L...,Sen. Elizabeth Warren (D-Mass.) campaigns in L...,"las vegas, discrimination laws, public educati...","federal court,broken nose,warren mass,immediat...","38, 65, 47"
2,0.407636,0.592364,https://www.ft.com/content/fe394284-35ae-11e9-...,1,Windstream debt battle opens up ‘Pandora’s Box’,Arkansas-based telecoms company now in trouble...,financial-times,2019-02-22T15:21:32Z,Addressing a room full of investors in Las Veg...,Addressing a room full of investors in Las Veg...,Addressing a room full of investors in Las Veg...,"las vegas, unpaid debts, around litigation, fe...","federal court,ceo words,unusual requirement,lo...","79, 59, 69"
3,0.491887,0.508113,https://in.reuters.com/article/kraft-heinz-sto...,1,Kraft Heinz shares fall 28 percent after write...,Kraft Heinz Co shares fell to a record low on ...,reuters,2019-02-22T19:52:51Z,Reuters Kraft Heinz Co shares fell to a re...,(Reuters) - Kraft Heinz Co shares fell to a re...,(Reuters) - Kraft Heinz Co shares fell to a re...,"food company, marquee brands, concern years, k...","traded paper,synergy realization,sluggish grow...","15, 28, 19"
4,0.499905,0.500095,https://www.cnbc.com/2019/02/22/boeing-goldman...,1,"Boeing, Goldman and three other Dow stocks are...",Just five of the stocks of the Dow 30 have car...,cnbc,2019-02-22T15:30:00Z,Just five of the stocks of the Dow 30 have car...,Just five of the stocks of the Dow 30 have car...,Just five of the stocks of the Dow 30 have car...,"record highs, home depot, double digits, year ...","home depot,year gains,top end,big runs,pattern...","11, 16, 13"
5,0.535152,0.464848,https://www.economist.com/prospero/2019/02/22/...,0,"An entertaining, if scattershot, look at the g...",“This Giant Beast That Is the Global Economy” ...,the-economist,2019-02-22T15:09:15Z,THE BIG SHORT 2015 managed to make entertain...,THE BIG SHORT (2015) managed to make entertain...,THE BIG SHORT (2015) managed to make entertain...,"financial crisis, screenplay oscar, celebrity ...","monetary policy,honest people,financial crisis...","93, 32, 33"
6,0.542251,0.457749,https://www.usatoday.com/story/sports/mlb/colu...,0,Padres still can't believe they landed Manny M...,San Diego was told that unless they were willi...,usa-today,2019-02-22T19:55:01Z,What I m Hearing USA TODAY Sports Bob Nighte...,What I'm Hearing: USA TODAY Sports' Bob Nighte...,What I'm Hearing: USA TODAY Sports' Bob Nighte...,"tony clark, mlb players, glorious days, franch...","machado play,guys machado,free agent,contract ...","13, 39, 2"
7,0.565315,0.434685,https://www.washingtonpost.com/news/powerpost/...,0,The Health 202: HHS head Azar will take a vict...,But the war has not been won.,the-washington-post,2019-02-22T13:30:13Z,President Trump speaks during an event about p...,THE PROGNOSIS\r\nPresident Trump speaks during...,President Trump speaks during an event about p...,"kaster health, tout efforts, opioid abuse, yea...","trump administration,white house,insurance opt...","70, 46, 67"
8,0.595690,0.404310,https://www.usatoday.com/story/opinion/2019/02...,0,Who's misusing your personal data? Why we need...,Here are a few

In [8]:
import frontPage as fp
fp.FrontPage(articleDB, trendingTermsDB)